## Link census tracts to toc tiers
* Find what proportion belongs within a particular tier

In [1]:
import intake
import numpy as np
import pandas as pd
import geopandas as gpd
import utils

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

bucket_name = 'city-planning-entitlements'

In [10]:
tracts = catalog.census_tracts.read().to_crs({'init':'epsg:2229'})

tracts = (
    tracts[['GEOID10', 'geometry']]
    .rename(columns = {'GEOID10': 'GEOID'})
    .assign(tract_sqft = tracts.geometry.area)
)


parcels = gpd.read_file(
    f"s3://{bucket_name}/gis/intermediate/toc_eligible_parcels_with_entitlements.geojson"
).to_crs({'init':'epsg:2229'})

parcels['max_tier'] = parcels.groupby('AIN')['TOC_Tier'].transform('max')
parcels = parcels[parcels.TOC_Tier == parcels.max_tier]
parcels = parcels.drop(columns = 'max_tier')

crosswalk_parcels_tracts = pd.read_parquet(f's3://{bucket_name}/data/crosswalk_parcels_tracts.parquet')

crosswalk_tracts_tiers = pd.read_parquet(f's3://{bucket_name}/data/crosswalk_tracts_toc_tiers.parquet')

In [12]:
# Join parcels to census tracts
m1 = pd.merge(parcels, crosswalk_parcels_tracts, on = 'AIN', how = 'inner')

In [13]:
# Join tracts to TOC tiers
m2 = pd.merge(tracts, crosswalk_tracts_tiers, on = 'GEOID', how = 'inner')

In [21]:
geoid = ['06037207301']
crosswalk_tracts_tiers[crosswalk_tracts_tiers.GEOID.isin(geoid)]

,GEOID,tiers_id,intersect_tier,intersect_sqft
425,06037207301,8,1,5.206929e-07
510,06037207301,7,1,5.206929e-07
871,06037207301,59,1,2.589551e-05
885,06037207301,58,1,2.589551e-05
915,06037207301,108,1,4.199262e-05
...,...,...,...,...
18760,06037207301,907,4,6.443748e-06
18764,06037207301,908,4,6.443748e-06
18768,06037207301,905,4,6.443748e-06
18772,06037207301,902,4,1.313348e-07


In [24]:
crosswalk_tracts_tiers.groupby('GEOID').agg({'intersect_sqft':'sum'})

,intersect_sqft
GEOID,
06037104310,1.246708e-06
06037104320,7.691627e-05
06037104401,3.070147e-05
06037104404,4.562750e-05
06037104500,4.654365e-06
...,...
06037980009,4.828417e-05
06037980010,6.327548e-04
06037980013,1.507591e-07


In [15]:
# Find what proportion of tract sqmi intersects with TOC tier
m2 = (m2
      .assign(pct_area = m2.intersect_sqft / m2.tract_sqft)
      .drop(columns = ['tract_sqft', 'intersect_sqft'])
)

In [18]:
m2.pct_area.describe()

count    1.895900e+04
mean     3.481762e-12
std      3.088087e-12
min      6.195921e-19
25%      7.857526e-13
50%      2.413908e-12
75%      6.081397e-12
max      9.091981e-12
Name: pct_area, dtype: float64